In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import gc
import os

In [2]:
user_id = 'ninaman'
#user_id = 'nightofwings'

domain = 'https://www.pttweb.cc'
url = '/user/{}?t=message'.format(user_id)
output_path = '/Users/zoe/Desktop/ptt_crawler/github_data/'

ex_lst = ['首頁', '上一頁', '下一頁', '尾頁']
usr_result = []
replace_ele = ['                ', '\n', '    ']

col_nm = ['article_id', 'board', 'author', 'article_title', 'article_date',
          'article_lnk', 'art_all', 'art_count', 
          'push_tag', 'push_floor', 'push_content', 'push_datetime_pre'] 
        
#page_dict = dict()
page_lst = []
finish_set = set()

In [3]:
def reply_parser(reply_soup):
    reply_lst = []
    for push in reply_soup.select('.e7-top'):
        push_content = push.select('.yellow--text')[-1].text.strip(':')
        #if cross year, 2019 > 2020, check month diff
        push_datetime_pre = push.select('.ml-3')[0].text
        push_floor = push.select('.e7-floor')[0].text
        push_tag = push.select('.mr-2')[-1].text
        reply_lst.append([push_content, push_datetime_pre, push_floor, push_tag])
    return pd.DataFrame(reply_lst, columns=['push_content', 'push_datetime_pre',
                                            'push_floor', 'push_tag'])

In [4]:
def page_parser(page_soup):
    rs_lst = []
    for x in page_soup.select('.thread-item'):
        reply_df = reply_parser(x)
        #article info
        reply_df['article_date'] = x.select('.thread-posttime')[0].text
        reply_df['article_lnk'] = x.a['href']
        reply_df['article_title'] = x.a.text
        
        #author = x.select('a .thread-list-author')[0].text
        ath_text = x.select('.thread-list-author')[0].text
        for _e in replace_ele:
            ath_text = ath_text.replace(_e, '')
        reply_df['author'] = ath_text.split('- 發表於')[0][3:]
        
        #split in batch
        #reply_df['article_id'] = article_lnk.split('/')[3]
        #reply_df['board'] = article_lnk.split('/')[2]
        #reply summary
        post_ele = x.select('.thread-popularity')[0].text.split(',')
        reply_df['art_all'] = post_ele[0].strip(' 留言')
        reply_df['art_count'] =  post_ele[1].strip(' 推噓總分: ')
        
        #temp_lst = [article_id, board, author, article_title, article_date,
        #            article_lnk, art_all, art_count, 
        #           push_tag, push_floor, push_content, push_datetime_pre]

        rs_lst.append(reply_df)
    return rs_lst

In [5]:
# rs = requests.get(domain + url)
# soup = BeautifulSoup(rs.content)
# board_distri = [[x.a.text, x.select('span.ml-2')[0].text] for x in soup.select('.e7-box')[2:]]
# board_df = pd.DataFrame(board_distri, columns=['board', 'all'])

In [6]:
while True:
    rs = requests.get(domain + url)
    soup = BeautifulSoup(rs.content)
    cur_page = [x for x in soup.select('.e7-active')][0].text
    
    page_result = page_parser(soup)
    usr_result.extend(page_result)
    #finish_set.update({url})
    
    #candidate page update
    #temp_dict = {x['href']: x.text.strip() for x in soup.select('.e7-pagination a') 
    #             if x.text not in ex_lst}
    
    page_lst.append([cur_page, url])
    #page_dict.update({:cur_page})
    #page_dict.update(temp_dict)
    #exec_urls = {x for x in page_dict if x not in finish_set}
    
    next_url = soup.find_all('a', text='下一頁')
    if not next_url:
        break
    
    print('current page is {}, url is {}'.format(cur_page, url))
    print(time.ctime())
    #get any url from avaliable url
    url = next_url[0]['href']
    gc.collect()

current page is 1, url is /user/ninaman?t=message
Sat Jan 11 23:20:44 2020
current page is 2, url is /user/ninaman?t=message&page=1
Sat Jan 11 23:20:45 2020
current page is 3, url is /user/ninaman?t=message&page=2
Sat Jan 11 23:20:47 2020
current page is 4, url is /user/ninaman?t=message&page=3
Sat Jan 11 23:20:49 2020
current page is 5, url is /user/ninaman?t=message&page=4
Sat Jan 11 23:20:51 2020
current page is 6, url is /user/ninaman?t=message&page=5
Sat Jan 11 23:20:52 2020
current page is 7, url is /user/ninaman?t=message&page=6
Sat Jan 11 23:20:54 2020
current page is 8, url is /user/ninaman?t=message&page=7
Sat Jan 11 23:20:55 2020
current page is 9, url is /user/ninaman?t=message&page=8
Sat Jan 11 23:20:57 2020
current page is 10, url is /user/ninaman?t=message&page=9
Sat Jan 11 23:20:59 2020
current page is 11, url is /user/ninaman?t=message&page=10
Sat Jan 11 23:21:01 2020
current page is 12, url is /user/ninaman?t=message&page=11
Sat Jan 11 23:21:04 2020
current page is 13

current page is 99, url is /user/ninaman?t=message&page=98
Sat Jan 11 23:23:26 2020
current page is 100, url is /user/ninaman?t=message&page=99
Sat Jan 11 23:23:27 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1554096240000000000&k2=544380531
Sat Jan 11 23:23:29 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1554027300000000000&k2=544148539
Sat Jan 11 23:23:30 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1553954940000000000&k2=543881623
Sat Jan 11 23:23:31 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1553915580000000000&k2=543776768
Sat Jan 11 23:23:33 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1553868660000000000&k2=543659933
Sat Jan 11 23:23:34 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1553775900000000000&k2=543314788
Sat Jan 11 23:23:36 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1553764560000000000&k2=543265329
Sat Jan 11 23:23:37 2020
current page is [ Δ

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551781620000000000&k2=536650717
Sat Jan 11 23:25:17 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551772560000000000&k2=536608417
Sat Jan 11 23:25:18 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551763140000000000&k2=536570391
Sat Jan 11 23:25:20 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551757020000000000&k2=536540294
Sat Jan 11 23:25:21 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551743640000000000&k2=536497056
Sat Jan 11 23:25:22 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551710820000000000&k2=536301798
Sat Jan 11 23:25:24 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551700980000000000&k2=536258879
Sat Jan 11 23:25:26 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551690960000000000&k2=536222658
Sat Jan 11 23:25:28 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=15516710400000

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550333940000000000&k2=531860501
Sat Jan 11 23:27:09 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550321760000000000&k2=531803163
Sat Jan 11 23:27:10 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550304600000000000&k2=531743435
Sat Jan 11 23:27:12 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550292900000000000&k2=531703852
Sat Jan 11 23:27:13 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550275620000000000&k2=531652564
Sat Jan 11 23:27:14 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550242440000000000&k2=531583569
Sat Jan 11 23:27:16 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550232420000000000&k2=531540717
Sat Jan 11 23:27:17 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550224860000000000&k2=531514115
Sat Jan 11 23:27:19 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=15502141200000

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1549027140000000000&k2=529799260
Sat Jan 11 23:28:56 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1549020720000000000&k2=529797747
Sat Jan 11 23:28:58 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1549011960000000000&k2=529796600
Sat Jan 11 23:28:59 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548993300000000000&k2=528237281
Sat Jan 11 23:29:01 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548948180000000000&k2=528147234
Sat Jan 11 23:29:02 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548940800000000000&k2=528115423
Sat Jan 11 23:29:04 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548930000000000000&k2=528074894
Sat Jan 11 23:29:06 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548912061000000000&k2=528012925
Sat Jan 11 23:29:07 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=15488952600000

...

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1474553992000000000&k2=31182101
Sat Jan 11 23:44:20 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1472837489000000000&k2=489352133
Sat Jan 11 23:44:21 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1472037800000000000&k2=390911606
Sat Jan 11 23:44:23 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1471416109000000000&k2=390810535
Sat Jan 11 23:44:25 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1470498979000000000&k2=492484495
Sat Jan 11 23:44:26 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1469954167000000000&k2=69565517
Sat Jan 11 23:44:28 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1469029408000000000&k2=301761696
Sat Jan 11 23:44:29 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1468829315000000000&k2=301720859
Sat Jan 11 23:44:31 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1468555406000000

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447825644000000000&k2=489077126
Sat Jan 11 23:46:12 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447735341000000000&k2=489022491
Sat Jan 11 23:46:14 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447720208000000000&k2=291498881
Sat Jan 11 23:46:15 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447638918000000000&k2=102646589
Sat Jan 11 23:46:17 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447587476000000000&k2=399840067
Sat Jan 11 23:46:18 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447517538000000000&k2=489021574
Sat Jan 11 23:46:20 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447383641000000000&k2=223356749
Sat Jan 11 23:46:21 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1447296136000000000&k2=262261204
Sat Jan 11 23:46:23 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=14471436350000

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1395995157000000000&k2=146293824
Sat Jan 11 23:48:04 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1395841639000000000&k2=146031602
Sat Jan 11 23:48:05 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1395719626000000000&k2=145726486
Sat Jan 11 23:48:07 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1395461121000000000&k2=30161965
Sat Jan 11 23:48:08 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1395114214000000000&k2=144354030
Sat Jan 11 23:48:10 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1394944462000000000&k2=68674751
Sat Jan 11 23:48:11 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1394861767000000000&k2=143730650
Sat Jan 11 23:48:13 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1394770944000000000&k2=487697765
Sat Jan 11 23:48:14 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1394419445000000

current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387924392000000000&k2=119524537
Sat Jan 11 23:49:54 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387853479000000000&k2=390393325
Sat Jan 11 23:49:56 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387723323000000000&k2=30028741
Sat Jan 11 23:49:57 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387605306000000000&k2=125580528
Sat Jan 11 23:49:59 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387506350000000000&k2=125263491
Sat Jan 11 23:50:00 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387442370000000000&k2=119097853
Sat Jan 11 23:50:01 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387404956000000000&k2=39184617
Sat Jan 11 23:50:03 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387336480000000000&k2=454123828
Sat Jan 11 23:50:05 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1387247770000000

current page is -27, url is /user/ninaman?t=message&page=-27
Sat Jan 11 23:52:11 2020
current page is -26, url is /user/ninaman?t=message&page=-26
Sat Jan 11 23:52:13 2020
current page is -25, url is /user/ninaman?t=message&page=-25
Sat Jan 11 23:52:14 2020
current page is -24, url is /user/ninaman?t=message&page=-24
Sat Jan 11 23:52:16 2020
current page is -23, url is /user/ninaman?t=message&page=-23
Sat Jan 11 23:52:17 2020
current page is -22, url is /user/ninaman?t=message&page=-22
Sat Jan 11 23:52:19 2020
current page is -21, url is /user/ninaman?t=message&page=-21
Sat Jan 11 23:52:21 2020
current page is -20, url is /user/ninaman?t=message&page=-20
Sat Jan 11 23:52:23 2020
current page is -19, url is /user/ninaman?t=message&page=-19
Sat Jan 11 23:52:24 2020
current page is -18, url is /user/ninaman?t=message&page=-18
Sat Jan 11 23:52:26 2020
current page is -17, url is /user/ninaman?t=message&page=-17
Sat Jan 11 23:52:27 2020
current page is -16, url is /user/ninaman?t=message&pa

In [27]:
gc.collect()
rs_df = pd.concat(usr_result)
#rs_df = pd.DataFrame(usr_result, columns=col_nm)
rs_df.replace(',', '，', regex=True, inplace=True)
rs_df.reset_index(drop=True, inplace=True)
rs_df_split = pd.concat([rs_df, rs_df.article_lnk.str.split('/', expand=True)[[2, 3]]], axis=1)
rs_df_split.rename({2:'board', 3:'article_id'}, axis=1, inplace=True)

rs_df_split.to_csv(output_path + '{}_reply.csv'.format(user_id), index=False)

In [28]:
rs_df_split.groupby('push_tag').agg({'board':'count'})

,board
push_tag,
→,12253
噓,1510
推,14410


In [9]:
#next & candidate union > over 200 pages have some isuues...
#narrow down to boards...
#rs_df.shape

(12100, 12)

In [9]:
board
push_tag	
→	12253
噓	1510
推	14410

留言數量: 28172
送出的『推』: 14412
送出的『→』: 12250
送出的『噓』: 1510
    
push_tag	
→	8279
噓	1699
推	12212
rs_df.groupby('push_tag').agg({'author':'count'})

,author
push_tag,
→,8279
噓,1699
推,12212
